In [3]:
import stanza
import pandas as pd
import re
import string
import ast

from SPARQLWrapper import SPARQLWrapper, JSON
import os
import locale
from datetime import datetime

import re
import pandas as pd
from rapidfuzz import fuzz as rapidfuzz_fuzz, process as rapidfuzz_process

## mapping.py

In [ ]:
def generate_mapping():
    stanza.download('id')
    nlp = stanza.Pipeline('id')
    templates = pd.read_csv('Lexid-be/question-sparql-template.csv')
    semantic_templates = pd.read_csv('Lexid-be/question-sparql-template-semantic.csv')
    question_parse = pd.read_csv('Lexid-be/question-parse-pattern.csv')
    # semantic_questions = pd.read_csv('Lexid-be/semantic-question-collection.csv').set_index('Question').fillna('')

    def set_query(query_index, is_semantic=False, **kwargs):
        if not is_semantic:
            if query_index == 0:
                return ''
            query_template = templates[templates['query_index'] == query_index]\
                                      .reset_index(drop=True)['query_template'][0]
            for key, value in kwargs.items():
                query_template = re.sub(key, value.title() if key != "ayat_num" else value,
                                        query_template)
            if query_index == 13:
                ayat = kwargs.get('ayat_num')
                if not ayat:
                    query_template = re.sub("ayat_num", "", query_template)
        else:
            query_template = semantic_templates[semantic_templates['query_index'] == query_index]\
                                                .reset_index(drop=True)['query_template'][0]
            for key, value in kwargs.items():
                query_template = re.sub(key, value.title(), query_template)
        return query_template

    def get_legal_target(target_idx, head_values, word_head, text_pos, word_values):
        target = ""
        try:
            if word_head[target_idx] == head_values:
                target_stop = target_idx
                # get the first number
                try:
                    first_num_index = text_pos.index("NUM", target_stop)
                    target_stop = first_num_index - 1
                except:
                    first_num_index = None
                # get second number
                try:
                    second_num_index = text_pos.index("NUM", first_num_index + 1)
                except:
                    second_num_index = None
                # get legal title
                for j in range(len(word_values[target_idx:target_stop])):
                    if word_values[target_idx+j][0] in ['/', ',']:
                        target += word_values[target_idx+j][0]
                        target += " " + word_values[target_idx+j][1:] if word_values[target_idx+j][1:] else ""
                    else:
                        target += word_values[target_idx+j]
                    if word_values[target_idx+j+1][0] not in ['/', ',']:
                        target += " "
                # handle legal number and year
                if first_num_index and second_num_index:
                    if word_values[first_num_index-1] == "nomor" and word_values[second_num_index-1] == "tahun":
                        target += word_values[first_num_index-1] + " " + word_values[first_num_index] \
                                + " " + word_values[second_num_index-1] + " " + word_values[second_num_index]
                    elif word_values[first_num_index-1] == "tahun" and word_values[second_num_index-1] == "nomor":
                        target += word_values[second_num_index-1] + " " + word_values[second_num_index] \
                                + " " + word_values[first_num_index-1] + " " + word_values[first_num_index]
                elif first_num_index:
                    target += word_values[first_num_index-1] + " " + word_values[first_num_index]
        except:
            pass
        return target.title()

    def get_question_type_and_legal_head(root, first, verb):
        target_row = question_parse[(question_parse['root']==root) &
                                    (question_parse['first']==first) &
                                    (question_parse['verb']==verb)].reset_index(drop=True)
        if len(target_row) == 0:
            return
        return {
            'q_type': target_row['q_type'][0],
            'legal_head': target_row['legal_head'][0],
            'legal_rel': target_row['legal_rel'][0]
        }

    def convert_passive_to_active_verb(verb):
        active_verb = ""
        if verb[2] in ['k', 't', 's', 'p'] and verb[3] not in ['k', 't', 's', 'p']:
            if verb[2] == "k":
                active_verb += "meng" + verb[3:]
            elif verb[2] == "t":
                active_verb += "men" + verb[3:]
            elif verb[2] == "s":
                active_verb += "meny" + verb[3:]
            else:
                active_verb += "mem" + verb[3:]
        else:
            if verb[2] in ['l', 'm', 'n', 'r', 'w', 'y']:
                active_verb += 'me' + verb[2:]
            elif verb[2] in ['c', 'd', 'j', 's', 't', 'z']:
                active_verb += 'men' + verb[2:]
            elif verb[2] in ['b', 'f', 'p', 'v']:
                active_verb += 'mem' + verb[2:]
            else:
                active_verb += "meng" + verb[2:]
        return active_verb

    def get_subject_label(target_idx, head_values, word_head, word_values):
        subject_label = ""
        if word_head[target_idx] == head_values:
            target_stop = len(word_values) if word_values[-1] not in string.punctuation else len(word_values) - 1
            for j in range(len(word_values[target_idx:target_stop])):
                if word_values[target_idx+j][0] in ['/', ',']:
                    subject_label += word_values[target_idx+j][0]
                    subject_label += " " + word_values[target_idx+j][1:] if word_values[target_idx+j][1:] else ""
                else:
                    subject_label += word_values[target_idx+j]
                try:
                    if word_values[target_idx+j+1][0] not in ['/', ',']:
                        subject_label += " "
                except:
                    pass

        return subject_label.strip()

    def contains_year(string):
        count = 0
        for char in string:
            if char.isdigit():
                count += 1
                if count == 4:
                    return True
        return False

    def mapping(text):
        # result_prefix = ""
        # try:
        #     legal_ref = semantic_questions['legal_ref'][text]
        #     pasal_ref = semantic_questions['pasal_ref'][text]
        #     ayat_ref = semantic_questions['ayat_ref'][text]
        #     result_prefix = f"Berdasarkan {pasal_ref} {ayat_ref} {legal_ref}:\n" if ayat_ref \
        #                     else f"Berdasarkan {pasal_ref} {legal_ref}:\n"
        #     if ayat_ref != '':
        #         return set_query(12, legal_title=legal_ref,
        #                          pasal_num=pasal_ref, ayat_num=ayat_ref), 12, result_prefix
        #     return set_query(11, legal_title=legal_ref,
        #                      pasal_num=pasal_ref, ayat_num=ayat_ref), 11, result_prefix
        # except:
        #     pass

        doc = nlp(text)
        if len(doc.sentences) != 1:
            return "", "", -1

        ## POS tagging & dependency parsing result
        words = doc.sentences[0].words
        text_pos = [word.upos for word in words]
        word_values = [word.text.lower() for word in words]
        word_deprel = [word.deprel for word in words]
        word_head = [word_values[word.head - 1] if word.head > 0 else '' for word in words]

        # get verb, root, first word
        verb = word_values[text_pos.index("VERB")].lower() if "VERB" in text_pos else ""
        root = word_values[word_deprel.index('root')].lower()
        first = word_values[0].lower()

        # non-semantic question param
        question_type = 0
        legal_head = ""
        legal_index = 0
        target_pasal = ""
        target_ayat = ""
        question_info = get_question_type_and_legal_head(root, first, verb)

        # semantic question param
        subject_label = ""
        act_label = ""
        alt_act = ""
        if question_info:
            question_type = question_info['q_type']
            legal_head = question_info['legal_head']
            legal_index = word_deprel.index(
                              question_info['legal_rel']
                          ) if question_info['legal_rel'] in word_deprel else 0
        else:
            if root == "apa" and first == "apa" and verb == "membuat":
                legal_head = verb
                legal_index = word_deprel.index('obj') if 'obj' in word_deprel else 0
                first_noun = word_values[text_pos.index("NOUN")] if 'NOUN' in  text_pos else ""
                if first_noun == "pertimbangan":
                    question_type = 4
                elif first_noun == "hukum":
                    question_type = 5
            elif root == "bunyi" and first == "bagaimana" and verb == "":
                if 'pasal' in word_values:
                    index_pasal = word_values.index("pasal")
                    if text_pos[index_pasal+1] == "NUM":
                        target_pasal += "pasal" + " " + word_values[index_pasal+1]
                if target_pasal != "":
                    if 'ayat' in word_values:
                        index_ayat = word_values.index("ayat")
                        if text_pos[index_ayat+1] == "NUM":
                            target_ayat += "ayat" + " " + word_values[index_ayat+1]
                    legal_head = root
                    legal_index = word_deprel.index('nmod') if 'nmod' in word_deprel else 0
                    if target_ayat != "":
                        question_type = 12
                    else:
                        question_type = 11
            elif root == "perubahan" and first == "bagaimana" and verb == "":
                if 'pasal' in word_values:
                    index_pasal = word_values.index("pasal")
                    if text_pos[index_pasal+1] == "NUM":
                        target_pasal += "pasal" + " " + word_values[index_pasal+1]
                if target_pasal != "":
                    if 'ayat' in word_values:
                        index_ayat = word_values.index("ayat")
                        if text_pos[index_ayat+1] == "NUM":
                            target_ayat += "ayat" + " " + word_values[index_ayat+1]
                    legal_head = root
                    question_type = 13
                    legal_index = word_deprel.index('nmod') if 'nmod' in word_deprel else 0
            else:
                if not contains_year(text): # if semantic content, question will not contain year
                    if root == "apa" and first == "apa" and verb != "":
                        if verb[:2] == "di":
                            act_label = convert_passive_to_active_verb(verb)
                            alt_act = verb
                            subject_rel = 'obl'
                        else:
                            act_label = verb
                            subject_rel = 'obj'
                        subject_index = word_deprel.index(subject_rel) if subject_rel in word_deprel else 0
                        subject_label =  get_subject_label(
                            subject_index, verb,
                            word_head, word_values
                        ) if subject_index != 0 else ''
                    elif root == "apa" and first == "apa" and verb == "":
                        first_noun = word_values[text_pos.index("NOUN")] if 'NOUN' in  text_pos else ""
                        if first_noun:
                            act_label = first_noun
                            print(word_deprel)
                            subject_rel = ['flat', 'nmod', 'det', 'compound']
                            for rel in subject_rel:
                                if rel not in word_deprel:
                                    continue
                                subject_index = word_deprel.index(rel, word_values.index(first_noun)+1)
                                subject_label =  get_subject_label(
                                    subject_index, first_noun,
                                    word_head, word_values
                                ) if subject_index != 0 else ''
                                if subject_label != '':
                                    break
                    else:
                        return '', "", -1
                else:
                    return '', "", -1

        legal_target = get_legal_target(legal_index, legal_head,
                                        word_head, text_pos, word_values) if legal_index != 0 else ''
        if legal_target != '':
            if target_pasal != '' and target_ayat != '':
                return set_query(question_type, False, legal_title=legal_target,
                                 pasal_num=target_pasal, ayat_num=target_ayat), '', question_type
            elif target_pasal != '':
                return set_query(question_type, False, legal_title=legal_target,
                                 pasal_num=target_pasal), '', question_type
            return set_query(question_type, False, legal_title=legal_target), '', question_type
        else:
            if subject_label != "" and act_label != "" and alt_act != "":
                return set_query(1, True, act_label=act_label, subject_label=subject_label),\
                       set_query(1, True, act_label=alt_act, subject_label=subject_label), 1
            elif subject_label != "" and act_label != "":
                return set_query(1, True, act_label=act_label, subject_label=subject_label), '', 1
            return '', '', -1

    return mapping

## execute_query.py

In [ ]:
def generate_get_query_result():
    # load_dotenv()
    sparql = SPARQLWrapper(os.environ.get('URL', 'http://localhost:7302/repositories/lexid'))
    # locale.setlocale(locale.LC_ALL, "id_ID.UTF-8")
    prefix = '''
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX wd: <https://www.wikidata.org/wiki/>
    PREFIX lexid-s: <https://w3id.org/lex-id/schema/>
    PREFIX lexid: <https://w3id.org/lex-id/data/>
    '''
    sparql.setReturnFormat(JSON)

    def get_result(query, question_type):
        sparql.setQuery(prefix + query)
        try:
            result = sparql.queryAndConvert()
            answer = ";\n".join(ans['answer']['value'] for ans in result['results']['bindings'] if ans)
            if not answer:
                return "Jawaban tidak ditemukan"
            return answer if question_type not in [2, 3] else datetime.strptime(answer, "%Y-%m-%d").strftime("%d %B %Y")
        except:
            return "error"
    return get_result

## alternative_mapping.py

In [ ]:
def generate_alternative_mapping():
    question_template_df = pd.read_csv("Lexid-be/question-sparql-template.csv")
    question_pattern = question_template_df["q_pattern"]
    semantic_template_df = pd.read_csv("Lexid-be/question-sparql-template-semantic.csv")
    semantic_questions_df = pd.read_csv('Lexid-be/semantic-question-collection.csv').fillna('')
    semantic_questions = list(semantic_questions_df['Question'])
    semantic_question_reference = semantic_questions_df.set_index('Question')
    peraturan_df = pd.read_csv("Lexid-be/valid-document-title.csv")
    peraturan_label = peraturan_df["Peraturan"]
    pasal_df = pd.read_csv("Lexid-be/valid-pasal-label.csv")
    pasal_label = pasal_df['pasal']
    pasal_with_ayat_df = pd.read_csv("Lexid-be/valid-pasal-with-ayat-label.csv")
    pasal_with_ayat = pasal_with_ayat_df['pasal_ayat']
    pasal_optional_ayat_df = pd.read_csv("Lexid-be/valid-pasal-optional-ayat-label.csv").fillna('')
    pasal_optional_ayat = pasal_optional_ayat_df['pasal_ayat']
    pasal_ayat_reference = pasal_optional_ayat_df.set_index('pasal_ayat')

    def set_query(query_index, is_semantic=False, **kwargs):
        template_used = semantic_template_df
        if not is_semantic:
            template_used = question_template_df

        query_template = template_used[template_used['query_index'] == query_index]\
                                              .reset_index(drop=True)['query_template'][0]

        for key, value in kwargs.items():
            query_template = re.sub(key, value, query_template)
        return query_template

    def alternative_mapping(input_text):
        ratio_metric = rapidfuzz_fuzz.ratio
        input_words = input_text.split(" ")
        # result_prefix = ""
        if len(input_words) <= 7:
            candidate_question_process = rapidfuzz_process.extract(
                input_text,
                semantic_questions,
                scorer=ratio_metric
            )[0][0]
            subject_label = semantic_question_reference['subject_label'][candidate_question_process]
            act_label = semantic_question_reference['act_label'][candidate_question_process]

            query_param = {
                'subject_label': subject_label,
                'act_label': act_label
            }
            query_index = 1

            return set_query(query_index, is_semantic=True,
                             **query_param), candidate_question_process, query_index

        question_part = " ".join(input_words[:4] + input_words[-1:])
        peraturan_part = " ".join(input_words[2:])
        pasal_ayat_part = " ".join(input_words[2:7])
        question_process = rapidfuzz_process.extract(
            question_part,
            question_pattern,
            scorer=ratio_metric,
            limit=4
        )
        peraturan_process = rapidfuzz_process.extract(
            peraturan_part,
            peraturan_label,
            scorer=ratio_metric,
            limit=5
        )
        peraturan = [p for p, _, _ in peraturan_process]
        candidate_list = {}
        for q, _, _ in question_process:
            query_index = int(question_template_df[question_template_df['q_pattern']==q]['query_index'])
            if query_index == 11 or query_index == 12 or query_index == 13:
                data_used = pasal_label
                pattern_replaced = "pasal_num" if query_index == 11 else "pasal_num ayat_num"
                if query_index == 12:
                    data_used = pasal_with_ayat
                elif query_index == 13:
                    data_used = pasal_optional_ayat
                pasal_ayat_process = rapidfuzz_process.extract(
                    pasal_ayat_part,
                    data_used,
                    scorer=ratio_metric,
                    limit=5
                )
                pasal_ayat = [a for a, _, _ in pasal_ayat_process]
                for title in peraturan:
                    question = re.sub('legal_title', title, q)
                    for pasal_ayat_num in pasal_ayat:
                        question_complete = re.sub(pattern_replaced, pasal_ayat_num, question)
                        candidate_list[question_complete] = {
                            'query_index': query_index,
                            'pasal_num': pasal_ayat_reference['pasal'][pasal_ayat_num],
                            'ayat_num': pasal_ayat_reference['ayat'][pasal_ayat_num],
                            'legal_title': title
                        }
            else:
                for title in peraturan:
                    question = re.sub('legal_title', title, q)
                    candidate_list[question] = {
                        'query_index': query_index,
                        'legal_title': title
                    }
        try:
            candidate_question_process = rapidfuzz_process.extract(
                input_text,
                [j for j in candidate_list.keys()] + semantic_questions,
                scorer=ratio_metric,
                score_cutoff=50)[0][0]
            query_index = candidate_list[candidate_question_process].pop('query_index')
            return set_query(query_index,
                            **candidate_list[candidate_question_process]
                            ), candidate_question_process, query_index
        except:
            try:
                subject_label = semantic_question_reference['subject_label'][candidate_question_process]
                act_label = semantic_question_reference['act_label'][candidate_question_process]

                query_param = {
                    'subject_label': subject_label,
                    'act_label': act_label
                }
                query_index = 1
                return set_query(query_index, is_semantic=True
                                 **query_param), candidate_question_process, query_index
            except:
                return "", "", -1

    return alternative_mapping

## main.py

In [ ]:
mapping = None
alternative_mapping = None
get_query_result = None
get_autocomplete_suggestion = None
error_query_result = "error"
not_found_query_result = "Jawaban tidak ditemukan"
suggestion_task = None

mapping = generate_mapping()
get_query_result = generate_get_query_result()
alternative_mapping = generate_alternative_mapping()

2024-05-24 10:31:33 INFO: Downloading default packages for language: id (Indonesian)...
2024-05-24 10:31:33 INFO: File exists: C:\Users\Rofif\stanza_resources\id\default.zip.
2024-05-24 10:31:36 INFO: Finished downloading models and saved to C:\Users\Rofif\stanza_resources.
2024-05-24 10:31:36 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2024-05-24 10:31:36 INFO: Use device: cpu
2024-05-24 10:31:36 INFO: Loading: tokenize
2024-05-24 10:31:36 INFO: Loading: pos
2024-05-24 10:31:36 INFO: Loading: lemma
2024-05-24 10:31:36 INFO: Loading: depparse
2024-05-24 10:31:37 INFO: Done loading processors!


In [ ]:
def get_answer(question: str):
    result = ""
    invalid = True
    query, alt_query, q_index = mapping(question)
    if query != "":
        alt_result = get_query_result(alt_query, q_index) if alt_query else ''
        query_result = get_query_result(query, q_index)
        if query_result != error_query_result:
            invalid = False
            if alt_result != "":
                if query_result == not_found_query_result:
                    result = alt_result
                else:
                    result = query_result
            else:
                result = query_result
    return {
        "invalid": invalid,
        "result": result,
        "query": query
    }

In [ ]:
def get_answer_v2(question: str):
    result = ""
    query, question, q_index= alternative_mapping(question)
    if query:
        query_result = get_query_result(query, q_index)
        if query_result != error_query_result:
            result = query_result
    return {
        "question": question,
        "result": result,
        "query": query
    }

## Create updated SPARQL Template



In [7]:
updated_sparql_templates = []

#1 Siapa yang menetapkan legal_title?
updated_sparql_templates.append(('Siapa yang menetapkan legal_title?', 1, '''
  SELECT DISTINCT ?answer
  WHERE {
      ?LegalDocument lexid-s:hasEnactionOfficial ?Official ;
              rdfs:label "legal_title"^^xsd:string .
      ?Official rdfs:label ?answer .
  }
  '''))

#2 Kapan legal_title diundangkan?
updated_sparql_templates.append(('Kapan legal_title diundangkan?', 2, '''
  SELECT DISTINCT ?answer
  WHERE
  {
    ?LegalDocument rdfs:label "legal_title"^^xsd:string ;
                  lexid-s:hasPromulgationDate ?answer .
  }
  '''))

#3 Kapan legal_title ditetapkan?
updated_sparql_templates.append(('Kapan legal_title ditetapkan?', 3, '''
  SELECT distinct ?answer
  WHERE
  {
    ?LegalDocument rdfs:label "legal_title"^^xsd:string ;
                    lexid-s:hasEnactionDate ?answer .
  }
  '''))

#4 Apa pertimbangan dalam membuat legal_title?
updated_sparql_templates.append(('Apa pertimbangan dalam membuat legal_title?', 4, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE
  {
    SELECT distinct ?value
    WHERE
    {
      ?LegalDocument rdfs:label "legal_title"^^xsd:string ;
                    lexid-s:considers ?value .
    }
    ORDER BY ?value
  }
  '''))

#5 Apa saja dasar hukum dalam membuat legal_title?
updated_sparql_templates.append(('Apa saja dasar hukum dalam membuat legal_title?', 5, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE
  {
    SELECT distinct ?value
    WHERE
    {
      ?LegalDocument rdfs:label "legal_title"^^xsd:string ;
                      lexid-s:hasLegalBasis ?LegalBasis .
      ?LegalBasis rdfs:label ?value .
    }
    ORDER BY ?LegalDocumentLabel ?value
  }
  '''))

#6 Apa saja peraturan yang berelasi dengan legal_title?
updated_sparql_templates.append(('Apa saja peraturan yang berelasi dengan legal_title?', 6, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE
  {
    SELECT distinct (coalesce(?label, ?ans) as ?value)
    WHERE
    {
      ?LegalDocument lexid-s:hasLegalBasis | lexid-s:implements | lexid-s:amends | lexid-s:repeals ?ans ;
                   rdfs:label "legal_title"^^xsd:string .
      ?ans a lexid-s:LegalDocument .
      OPTIONAL {
          ?ans rdfs:label ?label .
      }
    }
    ORDER BY ?LegalDocumentLabel ?value
  }
  '''))

#7 Peraturan manakah yang diubah oleh legal_title?
updated_sparql_templates.append(('Peraturan manakah yang diubah oleh legal_title?', 7, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE
  {
    SELECT distinct (coalesce(?label , ?ans) as ?value)
    WHERE {
        ?LegalDocument a lexid-s:LegalDocument ;
                      rdfs:label "legal_title"^^xsd:string ;
                      lexid-s:amends ?ans .
        ?ans a lexid-s:LegalDocument .
        OPTIONAL {
            ?ans rdfs:label ?label .
        }
    }
    ORDER BY ?value
  }
  '''))

#8 Peraturan manakah yang dicabut oleh legal_title?
updated_sparql_templates.append(('Peraturan manakah yang dicabut oleh legal_title?', 8, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE
  {
    SELECT distinct (coalesce(?label , ?ans) as ?value)
    WHERE {
        ?LegalDocument a lexid-s:LegalDocument ;
                      rdfs:label "legal_title"^^xsd:string ;
                      lexid-s:repeals ?ans .
        ?ans a lexid-s:LegalDocument .
        OPTIONAL {
            ?ans rdfs:label ?label .
        }
    }
    ORDER BY ?value
  }
  '''))

#9 Apa saja Bab yang dibahas dalam legal_title?
updated_sparql_templates.append(('Apa saja Bab yang dibahas dalam legal_title?', 9, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE {
      SELECT distinct (concat(?contentLabel, ": ", ?contentName) as ?value)
      WHERE {
          ?LegalDocument a lexid-s:LegalDocument ;
              rdfs:label "legal_title"^^xsd:string ;
              lexid-s:hasContent/lexid-s:hasPart* [
                  a lexid-s:Chapter ;
                  rdfs:label ?contentLabel ;
                  lexid-s:name ?contentName ;
              ] .
      }
      ORDER BY ?value
  }
  '''))

#10 Berapa jumlah Pasal yang diatur dalam legal_title?
updated_sparql_templates.append(('Berapa jumlah Pasal yang diatur dalam legal_title?', 10, '''
  SELECT (count(*) as ?answer)
  WHERE {
      ?LegalDocument a lexid-s:LegalDocument ;
                    rdfs:label "legal_title"^^xsd:string ;
                    lexid-s:hasContent/lexid-s:hasPart*/a lexid-s:Article .
  }
  '''))

#11 Bagaimana bunyi pasal_num dalam legal_title?
updated_sparql_templates.append(('Bagaimana bunyi pasal_num dalam legal_title?', 11, '''
  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE {
      SELECT distinct (concat(coalesce(?sectionName, ""), " ", ?ans) as ?value)
      WHERE {
          ?LegalDocument a lexid-s:LegalDocument ;
                        rdfs:label "legal_title"^^xsd:string ;
                        lexid-s:hasContent/lexid-s:hasPart* ?article .
          ?article a lexid-s:Article ;
                  rdfs:label "pasal_num"^^xsd:string .
          {
              {
                  ?article lexid-s:hasPart [
                      a lexid-s:Section ;
                      lexid-s:name ?sectionName ;
                      dct:description ?ans ;
                  ] .
              }
              UNION
              {
                  ?article dct:description ?ans .
              }
          }
      }
      ORDER BY ?value
  }
  '''))

#12 Bagaimana bunyi pasal_num ayat_num dalam legal_title?
updated_sparql_templates.append(('Bagaimana bunyi pasal_num ayat_num dalam legal_title?', 12, '''
  SELECT distinct ?answer
  WHERE {
      ?LegalDocument a lexid-s:LegalDocument ;
      rdfs:label "legal_title"^^xsd:string ;
      lexid-s:hasContent/lexid-s:hasPart* [
          a lexid-s:Article ;
          rdfs:label "pasal_num"^^xsd:string ;
          lexid-s:hasPart [
              a lexid-s:Section ;
              rdfs:label "ayat_num"^^xsd:string ;
              dct:description ?answer ;
          ] ;
      ] .
  }
  '''))

#13 Bagaimana perubahan bunyi pasal_num (ayat_num) dalam legal_title?
updated_sparql_templates.append(('Bagaimana perubahan bunyi pasal_num (ayat_num) dalam legal_title?', 13, '''
  SELECT distinct (concat('Tahun ', STR(?year), ' Nomor ', ?number, ': ', ?GroupConcat) as ?answer)
  WHERE
  {
      SELECT distinct ?year ?number (lcase(group_concat(distinct ?ValueWithPasalNum; separator = ";")) as ?GroupConcat)
      WHERE
      {
        {
              SELECT distinct ?year ?number (?larticle as ?Pasal_Num) (COALESCE(?lsection, "") as ?Ayat_Num)
              (lcase(IF(BOUND(?lsection),
                          concat(?larticle, " ", coalesce(?lsection, "")),
                          ?larticle))
                      as ?lcontent)
              (concat(?lcontent, ': ', ?value) as ?ValueWithPasalNum)
            WHERE
              {
                  {
                      SELECT distinct ?article ?year ?number
                      WHERE
                      {
                          ?LegalDocument a lexid-s:LegalDocument;
                              rdfs:label "legal_title"^^xsd:string ;
                              lexid-s:hasContent/lexid-s:hasPart* ?parent ;
                              lexid-s:amendedBy [
                                lexid-s:hasContent/lexid-s:modifies ?modification;
                                lexid-s:regulationYear ?year ;
                                lexid-s:regulationNumber ?number ;
                              ] .
                          ?modification lexid-s:hasModificationTarget ?parent ;
                                      lexid-s:hasModificationContent [ lexid-s:hasPart* ?article ;] .
                          ?article a lexid-s:Article ;
                      }
                  }

              BIND("pasal_num"^^xsd:string as ?larticle)
              ?article rdfs:label ?larticle .
              {
                {
                  ?article lexid-s:hasPart [
                    a lexid-s:Section ;
                    rdfs:label ?lsection ;
                    dct:description ?value ;
                  ] .
                }
                UNION
                {
                  ?article dct:description ?value .
                }
              }
          }
          ORDER BY ?year ?number ?Pasal_Num ?Ayat_Num ?value
        }
      }
      GROUP BY ?year ?number
      ORDER BY desc(?year) desc(?number) ?GroupConcat
  }
  '''))

#14 Apa saja pasal yang dihapus dalam legal_title?
updated_sparql_templates.append(('Apa saja pasal yang dihapus dalam legal_title?', 14, '''
  SELECT distinct (group_concat(distinct ?value; separator = ";") as ?answer)
  WHERE {
   ?LegalDocument a lexid-s:LegalDocument;
      rdfs:label "legal_title"^^xsd:string;
      lexid-s:amendedBy/lexid-s:hasContent/lexid-s:hasPart*/lexid-s:deletes ?deleted .
      ?deleted a lexid-s:Article ;
        rdfs:label ?value .
  }
  ORDER BY ?value
  '''))

#15 Apa saja pasal yang ditambahkan dalam legal_title?
updated_sparql_templates.append(('Apa saja pasal yang ditambahkan dalam legal_title?', 15, '''
  SELECT distinct (group_concat(distinct ?value; separator = ";") as ?answer)
  WHERE {
    ?LegalDocument a lexid-s:LegalDocument;
      rdfs:label "legal_title"^^xsd:string;
      lexid-s:amendedBy/lexid-s:hasContent/lexid-s:hasPart*/lexid-s:adds/lexid-s:hasAdditionContent ?content .
      ?content a lexid-s:Article;
            rdfs:label ?value .
  }
  ORDER BY ?value
  '''))

#16 Apakah legal_title mengalami amendemen?
updated_sparql_templates.append(('Apakah legal_title mengalami amendemen?', 16, '''
  SELECT (IF(count(distinct ?LegalDocument) > 0, "Ya, sudah diamendemen", "Tidak, tidak diamendemen") as ?answer)
  WHERE {
      ?LegalDocument a lexid-s:LegalDocument ;
        lexid-s:amends ?LegalDocumentAmended .
      ?LegalDocumentAmended a lexid-s:LegalDocument ;
        rdfs:label "legal_title"^^xsd:string .
  }
  '''))

#17 Apakah legal_title masih berlaku?
updated_sparql_templates.append(('Apakah legal_title masih berlaku?', 17, '''
  SELECT (IF(count(distinct ?LegalDocument) > 0, "Tidak berlaku lagi", "Ya, masih berlaku") as ?answer)
  WHERE {
      ?LegalDocument a lexid-s:LegalDocument ;
        lexid-s:repeals ?LegalDocumentRepealed .
      ?LegalDocumentRepealed a lexid-s:LegalDocument ;
        rdfs:label "legal_title"^^xsd:string .
  }
  '''))

#18 Apa saja pasal yang diubah dalam legal_title?
updated_sparql_templates.append(('Apa saja pasal yang diubah dalam legal_title?', 18, '''
  SELECT distinct (group_concat(distinct ?value; separator = ";") as ?answer)
  WHERE {
    ?LegalDocument a lexid-s:LegalDocument;
      rdfs:label "legal_title"^^xsd:string;
      lexid-s:amendedBy/lexid-s:hasContent/lexid-s:hasPart*/lexid-s:modifies/lexid-s:hasModificationContent ?content .
      ?content a lexid-s:Article;
            rdfs:label ?value .
  }
  ORDER BY ?value
  '''))

updated_sparql_templates_df = pd.DataFrame(data=updated_sparql_templates, columns=['q_pattern', 'query_index', 'query_template'])
updated_sparql_templates_df.to_csv("Lexid-be/question-sparql-template.csv", index=False)

## load dataset

In [ ]:
test_dataset = pd.read_csv('dataset/test_tesaurus.csv')
test_dataset

,question,answer,ner,legal_document,query,template_index,pasal_num,ayat_num
0,Apakah Peraturan Lembaga Kebijakan Pengadaan B...,Tidak berlaku lagi,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...",https://w3id.org/lex-id/data/Peraturan_LKPP_20...,\n SELECT (IF(count(distinct ?LegalDocument) ...,T17,NaN,NaN
1,Peraturan Menteri Kehutanan Republik Indonesia...,45,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Hut_2010_45,\n SELECT (count(*) as ?answer)\n WHERE {\n ...,T10,NaN,NaN
2,Peraturan Menteri Perdagangan Republik Indones...,30,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Dag_2017_84,\n SELECT (count(*) as ?answer)\n WHERE {\n ...,T10,NaN,NaN
3,Sudahkah Peraturan Menteri Keuangan Republik I...,"Ya, sudah diamendemen","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Keu_2012_190,\n SELECT (IF(count(distinct ?LegalDocument) ...,T16,NaN,NaN
4,Pejabat yang meratifikasi keabsahan hukum dari...,Budi Karya Sumadi,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",https://w3id.org/lex-id/data/Permen_Hub_2017_119,\n SELECT DISTINCT ?answer\n WHERE {\n ...,T1,NaN,NaN
...,...,...,...,...,...,...,...,...
8313,Peraturan Menteri Keuangan Republik Indonesia ...,Pasal 27A,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Keu_2010_109,\n SELECT distinct (group_concat(distinct ?va...,T15,NaN,NaN
8314,Kapan Peraturan Menteri Ketenagakerjaan Republ...,2019-10-18,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Naker_2019_19,\n SELECT distinct ?answer\n WHERE\n {\n ...,T3,NaN,NaN
8315,Sudahkah Peraturan Menteri Perindustrian Repub...,"Ya, sudah diamendemen","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]",https://w3id.org/lex-id/data/Permen_Perin_2015_34,\n SELECT (IF(count(distinct ?LegalDocument) ...,T16,NaN,NaN
8316,Sebutkan ketentuan hukum yang diamendemen sete...,Peraturan Menteri Keuangan Republik Indonesia ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",https://w3id.org/lex-id/data/Permen_Keu_2010_84,\n SELECT distinct (group_concat(?value; sepa...,T7,NaN,NaN


In [ ]:
test_dataset.loc[4402]

question          Sebutkan ketentuan-ketentuan hukum yang dihapu...
answer            Peraturan Menteri Keuangan Republik Indonesia ...
ner                [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
legal_document     https://w3id.org/lex-id/data/Permen_Keu_2014_226
query             \n  SELECT distinct (group_concat(?value; sepa...
template_index                                                   T8
pasal_num                                                       NaN
ayat_num                                                        NaN
Name: 4402, dtype: object

## Testing

In [ ]:
label2id = {"T1": 0, "T2": 1, "T3": 2, "T4": 3,
            "T5": 4, "T6": 5, "T7": 6, "T8": 7, "T9": 8,
            "T10": 9, "T11": 10, "T12": 11, "T13": 12,
            "T14": 13, "T15": 14, "T16": 15, "T17": 16, "T18": 17}

# Test
questions = test_dataset['question']
ground_truth = test_dataset['answer']
template_label = test_dataset['template_index']

true_pred = [0 for i in range(18)]
false_pred = [0 for i in range(18)]

true_recaps = []
false_recaps = []

for loop in range(84):
  if not os.path.exists(f"Lexid-be/recap/recap_{loop}.csv"):
    if loop != 0:
      recap_df = pd.read_csv(f'Lexid-be/recap/recap_{max(loop-1, 0)}.csv')
      true_pred = ast.literal_eval(recap_df['true_pred'][0])
      false_pred = ast.literal_eval(recap_df['false_pred'][0])
      true_recaps = ast.literal_eval(recap_df['true_recaps'][0])
      false_recaps = ast.literal_eval(recap_df['false_recaps'][0])
    for i in range(loop * 100, min(loop * 100 + 100, 8318)):
      answer = get_answer(questions[i])
      if answer['invalid']:
        answer = get_answer_v2(questions[i])

      if ground_truth[i] == answer['result']:
        true_pred[label2id[template_label[i]]] += 1
        true_recaps.append((questions[i], answer['query'], ground_truth[i], answer['result']))
      else:
        false_pred[label2id[template_label[i]]] += 1
        false_recaps.append((questions[i], answer['query'], ground_truth[i], answer['result']))
    recap_df = pd.DataFrame(data={
        'true_pred': [true_pred],
        'false_pred': [false_pred],
        'true_recaps': [true_recaps],
        'false_recaps': [false_recaps]
    })
    recap_df.to_csv(f"Lexid-be/recap/recap_{loop}.csv", index=False)

recap_df = pd.read_csv(f'Lexid-be/recap/recap_83.csv')
true_pred = ast.literal_eval(recap_df['true_pred'][0])
false_pred = ast.literal_eval(recap_df['false_pred'][0])
true_recaps = ast.literal_eval(recap_df['true_recaps'][0])
false_recaps = ast.literal_eval(recap_df['false_recaps'][0])

# acc is amount data, acc value
acc = [((true_pred[i] + false_pred[i]), true_pred[i] / (true_pred[i] + false_pred[i])) for i in range(18)]

# Q is for Ninggar's and Handi's way of collecting Template by its bigger term
# acc_q is amount data, acc_q value
acc_q = []
upper_fraction = 0
lower_fraction = 0

# Q1: T1, T2, T3, T4
q1 = [0, 1, 2, 3]
for i in q1:
  upper_fraction += true_pred[i]
  lower_fraction += true_pred[i] + false_pred[i]
acc_q.append((lower_fraction, upper_fraction / lower_fraction))

# Q2: T5, T6, T7, T8
q2 = [4, 5, 6, 7]
for i in q2:
  upper_fraction += true_pred[i]
  lower_fraction += true_pred[i] + false_pred[i]
acc_q.append((lower_fraction, upper_fraction / lower_fraction))

# Q3: T9, T10
q3 = [8, 9]
for i in q3:
  upper_fraction += true_pred[i]
  lower_fraction += true_pred[i] + false_pred[i]
acc_q.append((lower_fraction, upper_fraction / lower_fraction))

# Q4: T11, T12
q4 = [10, 11]
for i in q4:
  upper_fraction += true_pred[i]
  lower_fraction += true_pred[i] + false_pred[i]
acc_q.append((lower_fraction, upper_fraction / lower_fraction))

# Q5: T13, T14, T15, T16, T17, T18
q5 = [12, 13, 14, 15, 16, 17]
for i in q5:
  upper_fraction += true_pred[i]
  lower_fraction += true_pred[i] + false_pred[i]
acc_q.append((lower_fraction, upper_fraction / lower_fraction))

display_labels = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18']

disp_df = pd.DataFrame(data=acc,
                        index=display_labels, columns=['Amount of Data', 'Accuracy'])
display(disp_df)

display_labels_q = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

disp_df_q = pd.DataFrame(data=acc_q,
                        index=display_labels_q, columns=['Amount of Data', 'Accuracy'])
display(disp_df_q)

,Amount of Data,Accuracy
T1,500,0.574000
T2,500,0.000000
T3,500,0.000000
T4,500,0.196000
T5,500,0.214000
T6,500,0.596000
T7,500,0.092000
T8,500,0.030000
T9,500,0.120000
T10,500,0.248000


,Amount of Data,Accuracy
Q1,2000,0.192500
Q2,4000,0.212750
Q3,5000,0.207000
Q4,6000,0.190667
Q5,8318,0.187425


In [ ]:
df_true_recaps = pd.DataFrame(true_recaps, columns=['question', 'query', 'gold', 'pred'])

save_path = 'dataset/'
if not os.path.exists(save_path):
  os.makedirs(save_path)
df_true_recaps.to_csv('dataset/handis_lexid_qa_app_true_recaps.csv', index=False)

In [ ]:
df_false_recaps = pd.DataFrame(false_recaps, columns=['question', 'query', 'gold', 'pred'])

save_path = 'dataset/'
if not os.path.exists(save_path):
  os.makedirs(save_path)
df_false_recaps.to_csv('dataset/handis_lexid_qa_app_false_recaps.csv', index=False)

# Run Single Data

In [ ]:
answer = get_answer('Apa saja Bab yang dibahas dalam Peraturan Menteri Pekerjaan Umum Dan Perumahan Rakyat Republik Indonesia Nomor 07 Tahun 2018?')
if answer['invalid']:
  answer = get_answer_v2('Apa saja Bab yang dibahas dalam Peraturan Menteri Pekerjaan Umum Dan Perumahan Rakyat Republik Indonesia Nomor 07 Tahun 2018?')
answer


  SELECT distinct (group_concat(?value; separator = ";") as ?answer)
  WHERE {
      SELECT distinct (concat(?contentLabel, ": ", ?contentName) as ?value)
      WHERE {
          ?LegalDocument a lexid-s:LegalDocument ;
              rdfs:label "legal_title"^^xsd:string ;
              lexid-s:hasContent/lexid-s:hasPart* [
                  a lexid-s:Chapter ;
                  rdfs:label ?contentLabel ;
                  lexid-s:name ?contentName ;
              ] .
      }
      ORDER BY ?value
  }
  


{'invalid': False,
 'result': 'BAB I : KETENTUAN UMUM: KETENTUAN UMUM;BAB II : BENTUK BSPS: BENTUK BSPS;BAB III : JENIS KEGIATAN DAN BESARAN BSPS: JENIS KEGIATAN DAN BESARAN BSPS;BAB IV : PENERIMA BSPS: PENERIMA BSPS;BAB V : PENYELENGGARAAN BSPS: PENYELENGGARAAN BSPS;BAB VI : PENGAWASAN DAN PENGENDALIAN: PENGAWASAN DAN PENGENDALIAN;BAB VII : KETENTUAN PERALIHAN: KETENTUAN PERALIHAN;BAB VIII : KETENTUAN PENUTUP: KETENTUAN PENUTUP',
 'query': '\n  SELECT distinct (group_concat(?value; separator = ";") as ?answer)\n  WHERE {\n      SELECT distinct (concat(?contentLabel, ": ", ?contentName) as ?value)\n      WHERE {\n          ?LegalDocument a lexid-s:LegalDocument ;\n              rdfs:label "Peraturan Menteri Pekerjaan Umum Dan Perumahan Rakyat Republik Indonesia Nomor 07 Tahun 2018"^^xsd:string ;\n              lexid-s:hasContent/lexid-s:hasPart* [\n                  a lexid-s:Chapter ;\n                  rdfs:label ?contentLabel ;\n                  lexid-s:name ?contentName ;\n       